In [60]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

tf.__version__

'2.16.2'

In [61]:
dataset = pd.read_csv('../../../Data/DL_Tutorial/Churn_Modelling.csv')
dataset.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [62]:
X = dataset.iloc[:, 3:-1].values
y = dataset.iloc[:, -1].values

In [63]:
print(X)
print(y)

[[619 'France' 'Female' ... 1 1 101348.88]
 [608 'Spain' 'Female' ... 0 1 112542.58]
 [502 'France' 'Female' ... 1 0 113931.57]
 ...
 [709 'France' 'Female' ... 0 1 42085.58]
 [772 'Germany' 'Male' ... 1 0 92888.52]
 [792 'France' 'Female' ... 1 0 38190.78]]
[1 0 1 ... 1 1 0]


In [64]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
X[:, 2] = le.fit_transform(X[:, 2])
print(X[:, 2])

[0 0 0 ... 0 1 0]


In [65]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers = [('encoder', OneHotEncoder(), [1])], remainder = 'passthrough')
X = np.array(ct.fit_transform(X))
print(X)

[[1.0 0.0 0.0 ... 1 1 101348.88]
 [0.0 0.0 1.0 ... 0 1 112542.58]
 [1.0 0.0 0.0 ... 1 0 113931.57]
 ...
 [1.0 0.0 0.0 ... 0 1 42085.58]
 [0.0 1.0 0.0 ... 1 0 92888.52]
 [1.0 0.0 0.0 ... 1 0 38190.78]]


In [66]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

In [67]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)


# Building the ANN

In [68]:
# ann = tf.keras.models.Sequential([
#     tf.keras.layers.Dense(units=6, activation='relu'),
#     tf.keras.layers.Dense(units=6, activation='relu'),
#     tf.keras.layers.Dense(units=1, activation='sigmoid')
# ])
ann = tf.keras.models.Sequential()
ann.add(tf.keras.layers.Dense(units = 6, activation = 'relu'))
ann.add(tf.keras.layers.Dense(units = 6, activation = 'relu'))
ann.add(tf.keras.layers.Dense(units = 1, activation = 'sigmoid'))


ann.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
ann.fit(X_train, y_train, batch_size=256, epochs=100, verbose=2)

Epoch 1/100
32/32 - 1s - 22ms/step - accuracy: 0.5599 - loss: 0.7196
Epoch 2/100
32/32 - 0s - 6ms/step - accuracy: 0.6156 - loss: 0.6608
Epoch 3/100
32/32 - 0s - 6ms/step - accuracy: 0.6769 - loss: 0.6188
Epoch 4/100
32/32 - 0s - 6ms/step - accuracy: 0.7291 - loss: 0.5768
Epoch 5/100
32/32 - 0s - 6ms/step - accuracy: 0.7719 - loss: 0.5370
Epoch 6/100
32/32 - 0s - 6ms/step - accuracy: 0.7912 - loss: 0.5030
Epoch 7/100
32/32 - 0s - 6ms/step - accuracy: 0.8000 - loss: 0.4776
Epoch 8/100
32/32 - 0s - 6ms/step - accuracy: 0.8018 - loss: 0.4599
Epoch 9/100
32/32 - 0s - 6ms/step - accuracy: 0.8037 - loss: 0.4481
Epoch 10/100
32/32 - 0s - 6ms/step - accuracy: 0.8039 - loss: 0.4409
Epoch 11/100
32/32 - 0s - 6ms/step - accuracy: 0.8054 - loss: 0.4366
Epoch 12/100
32/32 - 0s - 6ms/step - accuracy: 0.8062 - loss: 0.4339
Epoch 13/100
32/32 - 0s - 6ms/step - accuracy: 0.8080 - loss: 0.4325
Epoch 14/100
32/32 - 0s - 6ms/step - accuracy: 0.8090 - loss: 0.4314
Epoch 15/100
32/32 - 0s - 6ms/step - accur

In [69]:
print(ann.predict(sc.fit_transform([[1, 0, 0, 600, 1, 40, 3, 60000, 2, 1, 1, 50000]]) > 0.5))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
[[0.15938172]]


In [70]:
y_pred = ann.predict(X_test)
y_pred = (y_pred > 0.5)
print(y_pred)
print(np.concatenate((y_pred.reshape(len(y_pred), 1), y_test.reshape(len(y_test), 1)), axis = 1))

63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step  
[[False]
 [False]
 [False]
 ...
 [False]
 [False]
 [False]]
[[0 0]
 [0 1]
 [0 0]
 ...
 [0 0]
 [0 0]
 [0 0]]


In [71]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

[[1531   64]
 [ 306   99]]


0.815